In [1]:
import numpy as np
import torch

from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_sample_weight
from numpy import random

import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

import os
from pathlib import Path
home = str(Path.home())

In [2]:
def LR(X_train, y_train, X_test, y_test):
    scaler  = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test  = scaler.transform(X_test)
    clf = LogisticRegressionCV(n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [3]:
def RF(X_train, y_train, X_test, y_test):
    clf = RandomForestClassifier(n_estimators=300, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_hat =    clf.predict(X_test)
    y_hat_pr = clf.predict_proba(X_test)[:, 1]
    
    return y_hat, y_hat_pr

In [158]:
factor = 0.8
nr_mean = 3
mean_tables = []
nr_samples=2000

attacks = ['fgsm', 'bim', 'pgd', 'aa', 'df', 'cw']
datasets = ['cifar10', 'cifar100', 'imagenet']
models = ['wrn28-10', 'vgg16', 'wrn50-2']
detectors = ['lid', 'multilid']

results = {}

for it in range(nr_mean):
    random_state = [21, 30, 65][it] # random.randint(100)
    print("random state: ", random_state)
    final_table = np.zeros((len(attacks), 2))
    base_path = os.path.join(home, 'workspace/multiLID/data/extract/run_{}/'.format(it+1))
    
    results[it] = {}
    
    for dataset in datasets:
        
        if dataset == 'imagenet':
            k=30
        else:
            k=20
        
        for model in models:
            
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
                
            if not dataset in results[it]:
                results[it][dataset] = {}
            if not model in results[it][dataset]:
                results[it][dataset][model] = {}
                
            #print("config: ", random_state, dataset, model)
            
            for detector in detectors:
                
                nor_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_normalos_8255.pt"))
                adv_fgsm  = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/fgsm/k{k}/{detector}_adverlos_8255.pt"))
                nor_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_normalos_8255.pt"))
                adv_bim   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/bim/k{k}/{detector}_adverlos_8255.pt"))
                nor_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_normalos_8255.pt"))
                adv_pgd   = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/pgd/k{k}/{detector}_adverlos_8255.pt"))
                nor_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_normalos_8255.pt"))
                adv_aa    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/aa/k{k}/{detector}_adverlos_8255.pt"))
                nor_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_normalos.pt"))
                adv_df    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/df/k{k}/{detector}_adverlos.pt"))
                nor_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_normalos.pt"))
                adv_cw    = torch.load(os.path.join(base_path,  f"{dataset}/{model}/{detector}/cw/k{k}/{detector}_adverlos.pt"))

                nor = [nor_fgsm, nor_bim, nor_pgd, nor_aa, nor_df, nor_cw]
                adv = [adv_fgsm, adv_bim, adv_pgd, adv_aa, adv_df, adv_cw]

                for rows in range(len(nor)):
                    X_nor = nor[rows]
                    X_adv = adv[rows]
                    if len(X_nor.shape) > 2: 
                        X_nor = nor[rows].reshape((nor[rows].shape[0], -1))
                        X_adv = adv[rows].reshape((adv[rows].shape[0], -1))

                    y_nor = np.zeros(X_nor.shape[0]).astype('int')
                    y_adv = np.ones(X_nor.shape[0]).astype('int')

                    x_train_n, x_test_n, y_train_n, y_test_n = train_test_split(X_nor, y_nor, test_size=1-factor, train_size=factor, random_state=random_state)
                    x_train_a, x_test_a, y_train_a, y_test_a = train_test_split(X_adv, y_adv, test_size=1-factor, train_size=factor, random_state=random_state)

                    X_train = np.concatenate((x_train_n, x_train_a))
                    y_train = np.concatenate((y_train_n, y_train_a))

                    X_test = np.concatenate((x_test_n, x_test_a))
                    y_test = np.concatenate((y_test_n, y_test_a))

                    if detector == 'lid':
                        y_hat, y_hat_pr = LR(X_train, y_train, X_test, y_test)
                    elif detector == 'multilid':
                        y_hat, y_hat_pr = RF(X_train, y_train, X_test, y_test)

                    auc = round(100*roc_auc_score(y_test, y_hat_pr), 2)
                    f1 =  round(100*f1_score(y_test, y_hat), 2)
     
                    if not detector in results[it][dataset][model]:
                        results[it][dataset][model][detector] = {}
                    
                    if not attacks[rows] in results[it][dataset][model][detector]: 
                        results[it][dataset][model][detector][attacks[rows]] = {}
                    
                    
                    if not 'auc' in results[it][dataset][model][detector][attacks[rows]]:
                        results[it][dataset][model][detector][attacks[rows]]['auc'] = {}
                    if not 'f1' in results[it][dataset][model][detector][attacks[rows]]:
                        results[it][dataset][model][detector][attacks[rows]]['f1' ] = {}
                        
                    results[it][dataset][model][detector][attacks[rows]]['auc'] = auc
                    results[it][dataset][model][detector][attacks[rows]]['f1'] = f1                    

                    # print(attacks[rows].upper(), '&' , auc, '&', f1)

random state:  21
random state:  30
random state:  65


# LID

In [159]:
lid_list_mean = []
for it_mean in range(nr_mean):
    lid_list = []
    for dataset in ['cifar10', 'cifar100', 'imagenet']:
        for model in ['wrn28-10', 'vgg16', 'wrn50-2']:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            tmp = np.zeros((len(attacks), 2))
            for it, val in enumerate(results[it_mean][dataset][model]['lid'].items()):
                #print(it, val)
                tmp[it, 0] = val[1]['auc'].copy()
                tmp[it, 1] = val[1]['f1'].copy()
            lid_list.append(tmp.copy())
    
    lid_list_mean.append(np.stack(lid_list))
lid_list_mean_res = np.stack(lid_list_mean)
lid_mean = np.mean(lid_list_mean_res, axis=0)
lid_var = np.var(lid_list_mean_res, axis=0)

In [172]:
latex_table = ""
for j in range(lid_mean.shape[1]):
    latex_table += "\\textbf{"+f"{attacks[j].upper()}"+"} & "
    for i in range(lid_mean.shape[0]):
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
    latex_table = latex_table[:-2] + "\\\\\n"

# Print the LaTeX table
print(latex_table)

\textbf{FGSM} & $97.16 \pm 1.69$ & $90.66 \pm 2.94$ & $88.10 \pm 0.27$ & $79.53 \pm 0.12$ & $98.18 \pm 0.49$ & $93.47 \pm 1.02$ & $77.97 \pm 1.21$ & $70.79 \pm 0.48$ & $71.38 \pm 9.94$ & $65.77 \pm 3.21$ \\
\textbf{BIM} & $87.09 \pm 0.23$ & $79.70 \pm 0.40$ & $93.00 \pm 2.11$ & $85.10 \pm 1.38$ & $95.63 \pm 0.92$ & $89.05 \pm 0.57$ & $81.57 \pm 3.68$ & $73.85 \pm 6.61$ & $94.45 \pm 0.29$ & $87.24 \pm 0.84$ \\
\textbf{PGD} & $89.25 \pm 0.61$ & $80.43 \pm 2.24$ & $90.40 \pm 0.32$ & $81.52 \pm 1.96$ & $97.82 \pm 0.07$ & $91.97 \pm 0.20$ & $85.27 \pm 1.23$ & $78.88 \pm 2.21$ & $96.21 \pm 0.41$ & $89.72 \pm 2.03$ \\
\textbf{AA} & $96.80 \pm 0.42$ & $91.07 \pm 0.85$ & $97.49 \pm 0.15$ & $92.42 \pm 0.34$ & $99.30 \pm 0.02$ & $95.66 \pm 0.57$ & $87.24 \pm 1.02$ & $78.96 \pm 0.07$ & $99.97 \pm 0.00$ & $99.45 \pm 0.11$ \\
\textbf{DF} & $95.38 \pm 0.04$ & $87.79 \pm 2.21$ & $86.48 \pm 0.26$ & $77.13 \pm 0.86$ & $57.04 \pm 0.61$ & $53.08 \pm 0.37$ & $54.41 \pm 0.03$ & $53.18 \pm 0.78$ & $54.67 \pm

In [ ]:
# LID

In [173]:
lid_list_mean = []
for it_mean in range(nr_mean):
    lid_list = []
    for dataset in ['cifar10', 'cifar100', 'imagenet']:
        for model in ['wrn28-10', 'vgg16', 'wrn50-2']:
            if dataset in ['cifar10', 'cifar100'] and model in ['wrn50-2']:
                continue
            if dataset in ['imagenet'] and model in  ['wrn28-10', 'vgg16']:
                continue
            
            tmp = np.zeros((len(attacks), 2))
            for it, val in enumerate(results[it_mean][dataset][model]['multilid'].items()):
                #print(it, val)
                tmp[it, 0] = val[1]['auc'].copy()
                tmp[it, 1] = val[1]['f1'].copy()
            lid_list.append(tmp.copy())
    
    lid_list_mean.append(np.stack(lid_list))
lid_list_mean_res = np.stack(lid_list_mean)
lid_mean = np.mean(lid_list_mean_res, axis=0)
lid_var = np.var(lid_list_mean_res, axis=0)

In [174]:
latex_table = ""
for j in range(lid_mean.shape[1]):
    latex_table += "\\textbf{"+f"{attacks[j].upper()}"+"} & "
    for i in range(lid_mean.shape[0]):
        mean = lid_mean[i, j, 0]
        variance = lid_var[i, j, 0]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
        mean = lid_mean[i, j, 1]
        variance = lid_var[i, j, 1]
        latex_table += f"${mean:.2f} \\pm {variance:.2f}$ & "
    latex_table = latex_table[:-2] + "\\\\\n"

# Print the LaTeX table
print(latex_table)

\textbf{FGSM} & $97.02 \pm 0.20$ & $91.13 \pm 0.77$ & $91.15 \pm 0.13$ & $83.05 \pm 1.95$ & $98.68 \pm 0.16$ & $95.43 \pm 1.23$ & $83.50 \pm 1.01$ & $77.04 \pm 0.54$ & $80.92 \pm 0.81$ & $74.15 \pm 0.14$ \\
\textbf{BIM} & $96.50 \pm 0.26$ & $89.97 \pm 0.52$ & $94.68 \pm 0.11$ & $88.03 \pm 1.06$ & $97.81 \pm 0.08$ & $91.80 \pm 0.08$ & $83.27 \pm 0.42$ & $75.75 \pm 2.19$ & $94.71 \pm 0.45$ & $87.88 \pm 0.22$ \\
\textbf{PGD} & $97.85 \pm 0.09$ & $93.45 \pm 0.05$ & $92.32 \pm 1.31$ & $84.59 \pm 1.92$ & $98.78 \pm 0.05$ & $94.86 \pm 1.17$ & $88.68 \pm 0.21$ & $82.09 \pm 0.67$ & $96.91 \pm 0.14$ & $91.20 \pm 0.15$ \\
\textbf{AA} & $99.64 \pm 0.00$ & $97.48 \pm 0.02$ & $98.76 \pm 0.05$ & $95.09 \pm 0.90$ & $99.88 \pm 0.00$ & $98.62 \pm 0.01$ & $91.60 \pm 0.26$ & $85.32 \pm 0.05$ & $99.88 \pm 0.01$ & $99.00 \pm 0.03$ \\
\textbf{DF} & $97.74 \pm 0.14$ & $94.47 \pm 0.11$ & $89.39 \pm 1.53$ & $84.78 \pm 1.10$ & $76.37 \pm 0.15$ & $71.80 \pm 1.52$ & $75.47 \pm 1.12$ & $70.50 \pm 0.94$ & $53.54 \pm